In [1]:
#- 패키지 임포트
import pandas as pd
import numpy as np
#import emoji # 이모지 제거용
import re
from soynlp.normalizer import * #반복되는 자음 제거
import time
from tqdm.notebook import tqdm #for문 진행률 
from time import sleep #for문 진행 시간 
from kiwipiepy import Kiwi #문장 분리2

from IPython.display import clear_output
from IPython import display # 램효율을 늘리기 위해 아웃풋 display를 지워줄 것.
from ipywidgets import Output

out = Output()
display.display(out)


Output()

In [2]:
''' 파일 불러오기 '''
df = pd.read_csv("../dataset/2022/본문_크롤링/나무문화_본문_2022_7월.csv", encoding='UTF-8')
# print(df.shape, type(df)) # 파일 구조 및 컬럼명 확인용
df['org_idx']= df.index # 인덱스 column 만들기: 문장 분리할 때 사용함
# print(df.columns) # org_idx 제대로 추가됐나 단순 확인용.
post_list = list(df['본문']) #자연어 전처리를 위해 본문을 리스트 형태로

In [3]:
''' 불용어 제거 '''
# post_list 양이 많을 경우 불용어 제거 중에 오류 나면 다시 리스트로 만드는 데 오래 걸려서 카피하는 것임. 안 하고 post_list로 진행해도 됨.
post_list2 = post_list.copy()
for i in range(len(post_list2)):
    post_list2[i] = re.sub('\n', ' ', string=str(post_list2[i])) # 줄바꿈을 띄어쓰기 하나로
    post_list2[i] = re.compile('[^|ㄱ-ㅎ|ㅏ-ㅣ|A-Z|a-z|0-9|가-힣]+').sub(' ',post_list2[i])
    post_list2[i] = repeat_normalize(post_list2[i], num_repeats=1)
	# ↑ ㅋㅋㅋ, ㅎㅎㅎ 등의 의미 없는 자음 삭제. num_repeats가 1이면 반복 글자 중 1개만 남기고 2면 2개만 남김.
    post_list2[i] = re.sub(r"^\s+|\s+$", "", post_list2[i]) # 문서 앞뒤 공백 제거
    post_list2[i] = post_list2[i].split() # 문서 내 공백(1개 이상) 기준으로 자르기
    post_list2[i] = ' '.join(post_list2[i]) # 공백 기준으로 나뉜 문서를 다시 1개의 공백을 두고 붙임.
	# ↑ 문서 내 다중 공백을 지우기 위함.]

df['cleaning'] = pd.DataFrame(post_list2) #불용어 제거된 본문을 데이터 프레임으로

In [4]:
df

,url,본문,org_idx,cleaning
0,https://blog.naver.com/yangyoseop91/222835899184,인트로\n살려달라고 써놓고 부랴부랴 수정하는 중... 이번주 주간일기도 마저 써야되...,0,인트로 살려달라고 써놓고 부랴부랴 수정하는 중 이번주 주간일기도 마저 써야되는디 ㅋ...
1,https://blog.naver.com/kuna611/222835896356,#25일급여 #삼성카드 #특판적금 #서울행복신협 #구즉신협 #카카오뱅크8%적금 #새...,1,25일급여 삼성카드 특판적금 서울행복신협 구즉신협 카카오뱅크8 적금 새빗켐 지대로사...
2,https://blog.naver.com/y_girok_/222835898249,안녕-!\n재미없었고 별 다른 일이 없어서\n조용했던 저번주와는 달리\n시끌벅적 우...,2,안녕 재미없었고 별 다른 일이 없어서 조용했던 저번주와는 달리 시끌벅적 우당탕탕 재...
3,https://blog.naver.com/roseyh78/222835897663,경주 켄싱턴리조트 퇴실 후 곧장 집으로 가자하면 어쩌나 했는데\n다행이 내가 가고 ...,3,경주 켄싱턴리조트 퇴실 후 곧장 집으로 가자하면 어쩌나 했는데 다행이 내가 가고 싶...
4,https://blog.naver.com/anuyheel/222835899414,수영구에서 뭐 재미있는 거 배울만한 게 있을까?\n새로운 것을 도전해보면서 문화 생...,4,수영구에서 뭐 재미있는 거 배울만한 게 있을까 새로운 것을 도전해보면서 문화 생활이...
...,...,...,...,...
2862,https://blog.naver.com/babjule/222796851268,안녕하세요^^ 오랜만에 맛과멋으로\n돌아온 꿈꾸는 블로거 로쉬니입니다~!!\n오늘은...,2862,안녕하세요 오랜만에 맛과멋으로 돌아온 꿈꾸는 블로거 로쉬니입니다 오늘은 이태원으로 ...
2863,https://blog.naver.com/tjdrud34/222796850531,베트남 푸꾸옥 투어 버스\n썬월드 혼똔섬 케이블카 외\nBEST5\n2022년 해외...,2863,베트남 푸꾸옥 투어 버스 썬월드 혼똔섬 케이블카 외 BEST5 2022년 해외여행지...
2864,https://blog.naver.com/dnpftlsh/222796844700,서울근교에서 당일여행으로 바다를 볼 수 있는 곳! 월미도에 다녀왔다.\n인천시 중구...,2864,서울근교에서 당일여행으로 바다를 볼 수 있는 곳 월미도에 다녀왔다 인천시 중구에 있...
2865,https://blog.naver.com/crazyboy/222796842010,○ 4332년 12월 15일 낮/옛 가야 땅을 지나며 시를 생각한다\n때로는\n쓰지...,2865,4332년 12월 15일 낮 옛 가야 땅을 지나며 시를 생각한다 때로는 쓰지 않아도...


In [5]:
''' 문장 분리 '''
kiwi = Kiwi() #kss보다 속도 빠름 
split_list = [kiwi.split_into_sents(df['cleaning'][i]) for i in tqdm(range(len(df['cleaning'])))]
df['split_list'] = split_list
df

  0%|          | 0/2867 [00:00<?, ?it/s]

,url,본문,org_idx,cleaning,split_list
0,https://blog.naver.com/yangyoseop91/222835899184,인트로\n살려달라고 써놓고 부랴부랴 수정하는 중... 이번주 주간일기도 마저 써야되...,0,인트로 살려달라고 써놓고 부랴부랴 수정하는 중 이번주 주간일기도 마저 써야되는디 ㅋ...,[(인트로 살려달라고 써놓고 부랴부랴 수정하는 중 이번주 주간일기도 마저 써야되는디...
1,https://blog.naver.com/kuna611/222835896356,#25일급여 #삼성카드 #특판적금 #서울행복신협 #구즉신협 #카카오뱅크8%적금 #새...,1,25일급여 삼성카드 특판적금 서울행복신협 구즉신협 카카오뱅크8 적금 새빗켐 지대로사...,[(25일급여 삼성카드 특판적금 서울행복신협 구즉신협 카카오뱅크8 적금 새빗켐 지대...
2,https://blog.naver.com/y_girok_/222835898249,안녕-!\n재미없었고 별 다른 일이 없어서\n조용했던 저번주와는 달리\n시끌벅적 우...,2,안녕 재미없었고 별 다른 일이 없어서 조용했던 저번주와는 달리 시끌벅적 우당탕탕 재...,[(안녕 재미없었고 별 다른 일이 없어서 조용했던 저번주와는 달리 시끌벅적 우당탕탕...
3,https://blog.naver.com/roseyh78/222835897663,경주 켄싱턴리조트 퇴실 후 곧장 집으로 가자하면 어쩌나 했는데\n다행이 내가 가고 ...,3,경주 켄싱턴리조트 퇴실 후 곧장 집으로 가자하면 어쩌나 했는데 다행이 내가 가고 싶...,[(경주 켄싱턴리조트 퇴실 후 곧장 집으로 가자하면 어쩌나 했는데 다행이 내가 가고...
4,https://blog.naver.com/anuyheel/222835899414,수영구에서 뭐 재미있는 거 배울만한 게 있을까?\n새로운 것을 도전해보면서 문화 생...,4,수영구에서 뭐 재미있는 거 배울만한 게 있을까 새로운 것을 도전해보면서 문화 생활이...,[(수영구에서 뭐 재미있는 거 배울만한 게 있을까 새로운 것을 도전해보면서 문화 생...
...,...,...,...,...,...
2862,https://blog.naver.com/babjule/222796851268,안녕하세요^^ 오랜만에 맛과멋으로\n돌아온 꿈꾸는 블로거 로쉬니입니다~!!\n오늘은...,2862,안녕하세요 오랜만에 맛과멋으로 돌아온 꿈꾸는 블로거 로쉬니입니다 오늘은 이태원으로 ...,"[(안녕하세요 오랜만에 맛과멋으로 돌아온 꿈꾸는 블로거 로쉬니입니다, 0, 35, ..."
2863,https://blog.naver.com/tjdrud34/222796850531,베트남 푸꾸옥 투어 버스\n썬월드 혼똔섬 케이블카 외\nBEST5\n2022년 해외...,2863,베트남 푸꾸옥 투어 버스 썬월드 혼똔섬 케이블카 외 BEST5 2022년 해외여행지...,[(베트남 푸꾸옥 투어 버스 썬월드 혼똔섬 케이블카 외 BEST5 2022년 해외여...
2864,https://blog.naver.com/dnpftlsh/222796844700,서울근교에서 당일여행으로 바다를 볼 수 있는 곳! 월미도에 다녀왔다.\n인천시 중구...,2864,서울근교에서 당일여행으로 바다를 볼 수 있는 곳 월미도에 다녀왔다 인천시 중구에 있...,"[(서울근교에서 당일여행으로 바다를 볼 수 있는 곳 월미도에 다녀왔다, 0, 36,..."
2865,https://blog.naver.com/crazyboy/222796842010,○ 4332년 12월 15일 낮/옛 가야 땅을 지나며 시를 생각한다\n때로는\n쓰지...,2865,4332년 12월 15일 낮 옛 가야 땅을 지나며 시를 생각한다 때로는 쓰지 않아도...,"[(4332년 12월 15일 낮 옛 가야 땅을 지나며 시를 생각한다, 0, 35, ..."


In [6]:
# 데이터프레임을 1000개 단위로 나누기
chunk_size = 10
num_chunks = len(df) // chunk_size + 1  # 나머지가 있는 경우 추가적인 청크 필요

dfs = {}
for i in range(num_chunks):
    start_idx = i * chunk_size
    end_idx = (i + 1) * chunk_size
    chunk_df = df[start_idx:end_idx]
    dfs[f"df_{i+1}"] = chunk_df

# 생성된 데이터프레임들 확인
for key, chunk_df in dfs.items():
    print(key)

df_1
df_2
df_3
df_4
df_5
df_6
df_7
df_8
df_9
df_10
df_11
df_12
df_13
df_14
df_15
df_16
df_17
df_18
df_19
df_20
df_21
df_22
df_23
df_24
df_25
df_26
df_27
df_28
df_29
df_30
df_31
df_32
df_33
df_34
df_35
df_36
df_37
df_38
df_39
df_40
df_41
df_42
df_43
df_44
df_45
df_46
df_47
df_48
df_49
df_50
df_51
df_52
df_53
df_54
df_55
df_56
df_57
df_58
df_59
df_60
df_61
df_62
df_63
df_64
df_65
df_66
df_67
df_68
df_69
df_70
df_71
df_72
df_73
df_74
df_75
df_76
df_77
df_78
df_79
df_80
df_81
df_82
df_83
df_84
df_85
df_86
df_87
df_88
df_89
df_90
df_91
df_92
df_93
df_94
df_95
df_96
df_97
df_98
df_99
df_100
df_101
df_102
df_103
df_104
df_105
df_106
df_107
df_108
df_109
df_110
df_111
df_112
df_113
df_114
df_115
df_116
df_117
df_118
df_119
df_120
df_121
df_122
df_123
df_124
df_125
df_126
df_127
df_128
df_129
df_130
df_131
df_132
df_133
df_134
df_135
df_136
df_137
df_138
df_139
df_140
df_141
df_142
df_143
df_144
df_145
df_146
df_147
df_148
df_149
df_150
df_151
df_152
df_153
df_154
df_155
df_156
df_157
df_158
df

In [ ]:
# ''' 분리된 문장 한 개를 한 row로 '''
# df2 = pd.DataFrame(columns = ['split_str', 'org_idx']) #용량을 줄이기 위해 새로운 데이터프레임으로

# ''' 문장 데이터 프레임 저장 '''
# df2.to_csv("../dataset/2022/본문_전처리/나무문화_본문_7월_전처리.csv", index=False, encoding='UTF-8')

In [14]:
import pandas as pd
df2 = pd.read_csv("../dataset/2022/본문_전처리/나무문화_본문_7월_전처리.csv", encoding='UTF-8')
df2

,split_str,org_idx
0,인트로 살려달라고 써놓고 부랴부랴 수정하는 중 이번주 주간일기도 마저 써야되는디 ㅋ...,0
1,귀갓길이다,0
2,17 07 22 DIMANCHE 튈르리 정원 오르세 미술관 오랑주리 미술관 샹젤리제...,0
3,그치만 오르세로 가기 위해 지났던 튈르리가 너무너무 아름다워서 모든 것을 용서하기로...,0
4,아침식사 먼저 보고 가세요,0
...,...,...
139486,염원이 담긴 연등 불국사는 볼거리들이 많습니다,2866
139487,특히 고찰답게 수목이 우거지고 조용한 곳이 많아 산책하기에도 좋은 사찰입니다,2866
139488,곳곳에 있는 호수와 연못은 나무그림자와 어우러져 한 폭의 그림이 됩니다,2866
139489,찌는듯한 무더위속에서 불국사를 돌아보니 땀이 비오듯 합니다 숙제를 한 듯 합니다,2866


In [8]:
dfs['df_1']['split_list'].index

RangeIndex(start=0, stop=10, step=1)

In [9]:
len(dfs)

287

In [10]:
for a in range(1,len(dfs)+1):
    for i in tqdm(dfs['df_%s'%a]['split_list'].index):
        for j in range(len(dfs['df_%s'%a]['split_list'][i])):
            split_str= {'split_str': dfs['df_%s'%a]['split_list'][i][j][0], 'org_idx': [i]}
            print(split_str['org_idx'])
            split_df= pd.DataFrame(split_str)
            df2 = pd.concat([df2,split_df])
            df2.reset_index(drop=True, inplace=True)
            display.clear_output(wait=True) # 출력지우기
        else:
            time.sleep(1)

df2.reset_index(drop=True, inplace=True)
df2.to_csv("../dataset/2022/본문_전처리/나무문화_본문_7월_전처리.csv", index=False, encoding='UTF-8')
df2

,split_str,org_idx
0,인트로 살려달라고 써놓고 부랴부랴 수정하는 중 이번주 주간일기도 마저 써야되는디 ㅋ...,0
1,귀갓길이다,0
2,17 07 22 DIMANCHE 튈르리 정원 오르세 미술관 오랑주리 미술관 샹젤리제...,0
3,그치만 오르세로 가기 위해 지났던 튈르리가 너무너무 아름다워서 모든 것을 용서하기로...,0
4,아침식사 먼저 보고 가세요,0
...,...,...
139486,염원이 담긴 연등 불국사는 볼거리들이 많습니다,2866
139487,특히 고찰답게 수목이 우거지고 조용한 곳이 많아 산책하기에도 좋은 사찰입니다,2866
139488,곳곳에 있는 호수와 연못은 나무그림자와 어우러져 한 폭의 그림이 됩니다,2866
139489,찌는듯한 무더위속에서 불국사를 돌아보니 땀이 비오듯 합니다 숙제를 한 듯 합니다,2866


In [4]:
df2.isnull().sum()

split_str    0
org_idx      0
dtype: int64

In [13]:
df2.dropna(axis=0, inplace=True)
df2.reset_index(drop=True, inplace=True)
df2.to_csv("../dataset/2022/본문_전처리/나무문화_본문_7월_전처리.csv", index=False, encoding='UTF-8')